In [21]:
import os
cwd = os.getcwd().replace('/vectorbt', '')
os.chdir(cwd)

In [22]:
import pandas as pd
import numpy as np
import vectorbt as vbt
vbt.settings.set_theme('dark')


### Upload dataset

In [23]:
dataset = pd.read_csv('data/dataset_5m_180day_tickers.csv' , index_col=0)
dataset.index = pd.to_datetime(dataset.index)

### Resample dataset

In [48]:
close = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('5Min')\
    .last()\
    .unstack()\
    .T

close_15m = dataset.copy().\
    groupby('symbol')['close']\
    .resample('15Min')\
    .last()\
    .unstack()\
    .T
close_1h = dataset.copy().\
    groupby('symbol')['close']\
    .resample('60Min')\
    .last()\
    .unstack()\
    .T

close_1d = dataset.copy().\
    groupby('symbol')['close']\
    .resample('1d')\
    .last()\
    .unstack()\
    .T

#### Working with indicators
* Create indicators for each timeframe
* Join tables using *align* method from pandas

In [49]:
rsi_15m = vbt.RSI.run(close_15m, window=14, short_name='RSI').rsi
rsi_15m, _ = rsi_15m.align(close, broadcast_axis=0,
                           method='ffill', join='right')

rsi_1h = vbt.RSI.run(close_1h, window=14, short_name='RSI').rsi
rsi_1h, _ = rsi_1h.align(close, broadcast_axis=0, method='ffill', join='right')

rsi_1d = vbt.RSI.run(close_1d, window=14, short_name='RSI').rsi
rsi_1d, _ = rsi_1d.align(close, broadcast_axis=0, method='ffill', join='right')

#### Define trading logic
* Create datafrmame shape with signal 0
* Define signal of trading logic

In [53]:
signal = rsi_15m.copy().applymap(lambda x: 0)
signal[
    (rsi_15m < 20) &
    (rsi_1h < 20) &
    (rsi_1d < 30)
] = 1
signal[
    (rsi_15m > 80) &
    (rsi_1h > 80) &
    (rsi_1d > 70)
] = -1


#### Backtesting strategy

In [54]:
entries = np.where(signal == 1, True, False)
exits = np.where(signal == -1, True, False)

entries_short = np.where(signal == -1, True, False)
exits_short = np.where(signal == 1, True, False)
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    short_entries=entries_short,
    short_exits=exits_short,
    fees=0.001  
)
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol',
                       yaxis_title='Max total return', height=400, width=1700)


FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…